In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [1]:
n_hidden = 35
lr = 0.01
epochs = 1000

string = "hello pytorch. how long can a rnn cell remember"
chars = "abcdeghijklmnopqrstuvwxyz ?!.,:;01"
char_list = [i for i in chars]
n_letters = len(char_list)

In [2]:
def string_to_onehot(string):
  start = np.zeros(shape=(len(char_list)), dtype=int)
  end = np.zeros(shape=(len(char_list)), dtype=int)
  start[-2] = 1
  end[-1] = 1
  for i in string:
    idx = char_list.index(i)
    zero = np.zeros(shape=len(char_list), dtype=int)
    zero[idx] = 1
    start = np.vstack([start, zero])
  output = np.vstack([start, end])
  return output

In [3]:
def onehot_to_word(onehot_1):
  onehot = torch.Tensor.numpy(onehot_1)
  return char_list[onehot.argmax()]

In [4]:
class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(RNN, self).__init__()

    self.input_size = input_size
    self.hidden_size = hidden_size
    self.output_size = output_size

    self.i2h = nn.Linear(input_size, hidden_size)
    self.h2h = nn.Linear(hidden_size, hidden_size)
    self.i2o = nn.Linear(hidden_size, output_size)
    self.act_fn = nn.Tanh()
  
  def forward(self, input, hidden):
    hidden = self.act_fn(self.i2h(input)+self.h2h(hidden))
    output = self.i2o(hidden)
    return output, hidden
  
  def init_hidden(self):
    return torch.zeros(1, self.hidden_size)

NameError: ignored

In [ ]:
rnn = RNN(n_letters, n_hidden, n_letters)

In [27]:
loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=lr)

NameError: ignored

In [8]:
one_hot = torch.from_numpy(string_to_onehot(string)).type_as(torch.FloatTensor())
one_hot.shape

torch.Size([49, 34])

In [9]:
for i in range(epochs):
  rnn.zero_grad()
  total_loss = 0
  hidden = rnn.init_hidden()

  for j in range(one_hot.size()[0]-1):# 49-1=48
    input = one_hot[j:j+1,:]
    target = one_hot[j+1]

    output, hidden = rnn.forward(input, hidden)
    loss = loss_func(output.view(-1), target.view(-1))
    total_loss += loss
    input = output
  
  total_loss.backward()
  optimizer.step()

  if i%10==0:
    print(total_loss)


tensor(1.9514, grad_fn=<AddBackward0>)
tensor(0.9217, grad_fn=<AddBackward0>)
tensor(0.5794, grad_fn=<AddBackward0>)
tensor(0.3661, grad_fn=<AddBackward0>)
tensor(0.2461, grad_fn=<AddBackward0>)
tensor(0.1862, grad_fn=<AddBackward0>)
tensor(0.1368, grad_fn=<AddBackward0>)
tensor(0.1184, grad_fn=<AddBackward0>)
tensor(0.1046, grad_fn=<AddBackward0>)
tensor(0.0863, grad_fn=<AddBackward0>)
tensor(0.0860, grad_fn=<AddBackward0>)
tensor(0.0681, grad_fn=<AddBackward0>)
tensor(0.0577, grad_fn=<AddBackward0>)
tensor(0.0562, grad_fn=<AddBackward0>)
tensor(0.0441, grad_fn=<AddBackward0>)
tensor(0.0461, grad_fn=<AddBackward0>)
tensor(0.0340, grad_fn=<AddBackward0>)
tensor(0.0290, grad_fn=<AddBackward0>)
tensor(0.0414, grad_fn=<AddBackward0>)
tensor(0.0332, grad_fn=<AddBackward0>)
tensor(0.0263, grad_fn=<AddBackward0>)
tensor(0.0221, grad_fn=<AddBackward0>)
tensor(0.0199, grad_fn=<AddBackward0>)
tensor(0.0211, grad_fn=<AddBackward0>)
tensor(0.0172, grad_fn=<AddBackward0>)
tensor(0.0153, grad_fn=<A

In [10]:
start = torch.zeros(1, len(char_list))
start[:, -2] = 1

with torch.no_grad():
  hidden = rnn.init_hidden()
  input = start
  output_string = ""

  for i in range(len(string)):
    output, hidden = rnn.forward(input, hidden)
    output_string += onehot_to_word(output.data)
    input = output

print(output_string)

hello pyo re rom eo oo no oo npcoo c  o na c. o


### Embedding
* 원핫인코딩은  배겉간의 내적이 0ㅇ이 나오기에, 문장간의 차이나 유사도를 구하는 것이 불가능함.  거기다가 하나의 요소가 추가최면 백터의 길이가 늘어나 모델이 무의미 해짐.
* 알파벳이나 단어 같은 기본단위 요소들을 일정하 길이를 갖는 벡터 공간에 투영하는 것.

In [12]:
!rm -r data
import os 

try:
  os.mkdir("./data")
except:
  pass

!wget https://raw.githubusercontent.com/dmlc/web-data/master/mxnet/tinyshakespeare/input.txt -P ./data

rm: cannot remove 'data': No such file or directory
--2021-04-28 12:58:04--  https://raw.githubusercontent.com/dmlc/web-data/master/mxnet/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘./data/input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.04s   

2021-04-28 12:58:05 (23.9 MB/s) - ‘./data/input.txt’ saved [1115394/1115394]



In [13]:
!pip install unidecode

     |████████████████████████████████| 245kB 8.3MB/s 


In [14]:
import unidecode
import string
import random
import re
import time, math

In [15]:
num_epochs = 2000
print_every = 100
plot_every = 10

# chunk에 대한 설명은 아래 함수정의하면서 하겠습니다.
chunk_len = 200

hidden_size = 100
batch_size = 1
num_layers = 1
embedding_size = 70
lr = 0.002

In [16]:
all_characters = string.printable
n_characters = len(all_characters)
print(all_characters)
print("Num Chars :", n_characters)

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	

Num Chars : 100


In [17]:
file = unidecode.unidecode(open('./data/input.txt').read())
file_len = len(file)
print("File len :", file_len)

File len : 1115394


In [18]:
def random_chunk():
  start_index = random.randint(0, file_len - chunk_len)
  end_index = start_index + chunk_len + 1
  return file[start_index:end_index]

In [19]:
print(random_chunk())

give my voice on Richard's side,
To bar my master's heirs in true descent,
God knows I will not do it, to the death.

CATESBY:
God keep your lordship in that gracious mind!

HASTINGS:
But I shall laugh


In [20]:
def char_tensor(string):
  tensor = torch.zeros(len(string)).long()
  for i in range(len(string)):
    tensor[i] = all_characters.index(string[i])
  return tensor

In [21]:
print(char_tensor('ssszz    hizz'))

tensor([28, 28, 28, 35, 35, 94, 94, 94, 94, 17, 18, 35, 35])


In [22]:
def random_training_set():
  chunk = random_chunk()
  inp = char_tensor(chunk[:-1])
  target = char_tensor(chunk[1:])
  return inp, target

In [23]:
class RNN(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers=1):
    super(RNN, self).__init__()
    self.input_size = input_size
    self.embedding_size = embedding_size
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.num_layers = num_layers
    
    self.encoder = nn.Embedding(input_size, embedding_size)
    self.rnn = nn.RNN(embedding_size, hidden_size, num_layers)
    self.decoder = nn.Linear(hidden_size, output_size)

  def forward(self, input, hidden):
    out = self.encoder(input.view(batch_size,-1))
    out, hidden = self.rnn(out, hidden)
    out = self.decoder(out.view(batch_size, -1))
    return out, hidden
    
  def init_hidden(self):
    hidden = torch.zeros(self.num_layers, batch_size, hidden_size)
    return hidden

In [ ]:
class GRU(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers=1):
    super(RNN, self).__init__()
    self.input_size = input_size
    self.embedding_size = embedding_size
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.num_layers = num_layers
    
    self.encoder = nn.Embedding(input_size, embedding_size)
    self.rnn = nn.GRU(embedding_size, hidden_size, num_layers)
    self.decoder = nn.Linear(hidden_size, output_size)

  def forward(self, input, hidden):
    out = self.encoder(input.view(batch_size,-1))
    out, hidden = self.rnn(out, hidden)
    out = self.decoder(out.view(batch_size, -1))
    return out, hidden
    
  def init_hidden(self):
    hidden = torch.zeros(self.num_layers, batch_size, hidden_size)
    return hidden

In [ ]:
class LSTM(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers=1):
    super(RNN, self).__init__()
    self.input_size = input_size
    self.embedding_size = embedding_size
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.num_layers = num_layers
    
    self.encoder = nn.Embedding(input_size, embedding_size)
    self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers)
    self.decoder = nn.Linear(hidden_size, output_size)

  def forward(self, input, hidden):
    out = self.encoder(input.view(batch_size,-1))
    out, (hidden, cell) = self.rnn(out, hidden)
    out = self.decoder(out.view(batch_size, -1))
    return out, hidden, cell
    
  def init_hidden(self):
    hidden = torch.zeros(self.num_layers, batch_size, hidden_size)
    cell = torch.zeros(self.num_layers, batch_size, hidden_size)
    return hidden

In [24]:
model = RNN(
    input_size=n_characters,
    embedding_size=embedding_size,
    hidden_size=hidden_size,
    output_size=n_characters,
    num_layers=2
)

In [31]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_func = nn.CrossEntropyLoss()

In [40]:
# 임의의 문자(start_str)로 시작하는 길이 200짜리 모방 글을 생성하는 코드입니다.
def test():
    start_str = "b"
    inp = char_tensor(start_str)
    hidden = model.init_hidden()
    x = inp

    print(start_str,end="")
    for i in range(200):
        output,hidden = model(x,hidden)

        # 여기서 max값을 사용하지 않고 multinomial을 사용하는 이유는 만약 max 값만 쓰는 경우에
        # 생성되는 텍스트가 다 the the the the the 이런식으로 나오기 때문입니다.
        # multinomial 함수를 통해 높은 값을 가지는 문자들중에 램덤하게 다음 글자를 뽑아내는 방식으로 자연스러운 텍스트를 생성해냅니다.
        output_dist = output.data.view(-1).div(0.8).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        predicted_char = all_characters[top_i]

        print(predicted_char,end="")

        x = char_tensor(predicted_char)

In [42]:
for i in range(num_epochs):
    # 랜덤한 텍스트 덩어리를 샘플링하고 이를 인덱스 텐서로 변환합니다. 
    inp,label = random_training_set()
    hidden = model.init_hidden()

    loss = torch.tensor([0]).type(torch.FloatTensor)
    optimizer.zero_grad()
    for j in range(chunk_len-1):
        x  = inp[j]
        y_ = label[j].unsqueeze(0).type(torch.LongTensor)
        y,hidden = model(x,hidden)
        loss += loss_func(y,y_)

    loss.backward()
    optimizer.step()
    
    if i % 100 == 0:
        print("\n",loss/chunk_len,"\n")
        test()
        print("\n","="*100)


 tensor([1.6685], grad_fn=<DivBackward0>) 

buch all rod,
And thy counted it;
And the vielligate in that be lord, when on thee with on that do is your hands,
No go seath, by my doth were if
of my stay, gone on your will comporture.

BALUCENTIO:


 tensor([1.4342], grad_fn=<DivBackward0>) 

by. Marce so right shape batty, your calls, me the meace,
I in come at of the fices, Wery and prayer say cornioate,
Uleases to that the oody own it was will gurdery lady.

SEMPROSE:
Snettin would seem 

 tensor([1.5031], grad_fn=<DivBackward0>) 

brouse to our post all, debiping the beside then in my subving of help fails he sear clound lord destreat and the sealy notting sift to tere that I to the good first dear's busine will amprails those s

 tensor([1.7917], grad_fn=<DivBackward0>) 

bry, I have make selven from thou have righting will nobsence bids speak this much lands me, deeps and go it lear'd how the compa sance the friends,
While of Bounwal thy have on contenger and pitized B

 tensor([1.